# 🧠 Labeling from Top Rule-Based Configs

This notebook prototypes labeling touch events as 'good bounces' using trades from top-performing rule-based strategies.

📦 **Setup paths and imports** — Load necessary modules and define file paths.

In [33]:
# ✅ Setup
import pandas as pd
import json
from pathlib import Path

RESULTS_DIR = Path('../results')
CONFIG_PATH = Path('../configs/top_10_configs.json')
TRADES_PATH = RESULTS_DIR / 'batch_backtest_results.csv'

📥 **Load top rule-based config IDs** — Read JSON file with the top filter configurations.

In [34]:
# ✅ Load top rule-based config IDs
with open(CONFIG_PATH) as f:
    top_configs = json.load(f)

top_config_ids = [cfg['Config_ID'] for cfg in top_configs]
print(f"Loaded {len(top_config_ids)} top configs:", top_config_ids)

Loaded 10 top configs: ['CFG_097', 'CFG_101', 'CFG_113', 'CFG_121', 'CFG_105', 'CFG_117', 'CFG_125', 'CFG_109', 'CFG_081', 'CFG_089']


📊 **Load rule-based trade results** — Load the CSV of all trades and filter for top configs.

In [35]:
# ✅ Load all trades and keep only those from top configs
all_trades = pd.read_csv(TRADES_PATH, parse_dates=['Entry Date', 'Exit Date'])
top_trades = all_trades[all_trades['Config_ID'].isin(top_config_ids)].copy()
print(f"Filtered to {len(top_trades)} trades from top rule-based configs.")
top_trades.head()

Filtered to 2092 trades from top rule-based configs.


Ticker Entry Date  Exit Date  Buy Price  Sell Price   Return %  \
11750   AAPL 2008-01-28 2008-02-04     130.01      131.65   1.261441   
11751   AAPL 2008-01-29 2008-02-05     131.54      129.36  -1.657291   
11752   AAPL 2008-02-07 2008-02-14     121.24      127.46   5.130320   
11753   AAPL 2008-10-09 2008-10-16      88.74      101.89  14.818571   
11754   AAPL 2010-08-27 2010-09-03     241.62      258.77   7.097922   

      Config_ID  USE_CONFIRMATION_CANDLE  USE_RSI_FILTER  \
11750   CFG_081                     True           False   
11751   CFG_081                     True           False   
11752   CFG_081                     True           False   
11753   CFG_081                     True           False   
11754   CFG_081                     True           False   

       USE_MACD_HIST_FILTER  USE_FORCE_INDEX_FILTER  USE_ATR_FILTER  \
11750                  True                   False           False   
11751                  True                   False           False   
11752                  True                   False           False   
11753                  True                   False           False   
11754                  True                   False           False   

       USE_MACD_DIVERGENCE  USE_TRAILING_EXIT  
11750                False              False  
11751                False              False  
11752                False              False  
11753                False              False  
11754                False              False

📈 **Label trades from top configs** — Add a new column `HighQuality` for trades from top configs.

In [36]:
# 🧪 Choose a sample ticker to prototype
sample_ticker = 'AAPL'
sample_trades = top_trades[top_trades['Ticker'] == sample_ticker]
print(f"Top trades for {sample_ticker}: {len(sample_trades)}")
sample_trades[['Entry Date', 'Return %']].head()

Top trades for AAPL: 1040


Entry Date   Return %
11750 2008-01-28   1.261441
11751 2008-01-29  -1.657291
11752 2008-02-07   5.130320
11753 2008-10-09  14.818571
11754 2010-08-27   7.097922

📤 **Export labeled dataset** — Save filtered trades with labels to a new CSV file.

In [37]:
# ✅ Load touch events (daily data) for selected ticker
df_daily = pd.read_csv(f"../stock_historical_information/daily/{sample_ticker}_daily.csv", parse_dates=['Date'])

# Simulate all rows as touch dates for now (could be refined later)
df_touch = df_daily[['Date']].copy()
df_touch['Label'] = df_touch['Date'].isin(sample_trades['Entry Date']).astype(int)
df_touch.head()

Date  Label
0 2007-01-03      0
1 2007-01-04      0
2 2007-01-05      0
3 2007-01-08      0
4 2007-01-09      0

🔍 **Preview output** — Show the first few labeled trades.

In [38]:
# 📊 Show label distribution
label_counts = df_touch['Label'].value_counts(normalize=True).rename_axis('Label').reset_index(name='Proportion')
label_counts

Label  Proportion
0      0    0.972493
1      1    0.027507